#Thanks to: https://pythonology.eu/a-rag-web-scraper-with-langchain-ollama-and-chroma/

Getting repo readme context, applied to:

1. https://github.com/JAlcocerT/Streamlit-MultiChat
2. https://github.com/qatrackplus/qatrackplus/

In [3]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install langchain-community


In [2]:
!pip show langchain

Name: langchain
Version: 0.3.25
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [ ]:
#brings your OPENAI_API_KEY
!source .env

In this step, we’re using **BeautifulSoup’s SoupStrainer** to focus on the “content-area” class, ensuring we only extract relevant information. 

The WebBaseLoader then fetches and processes the specified URL.

In [6]:
# class="markdown-body entry-content container-lg"

import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("markdown-body entry-content container-lg"))
loader = WebBaseLoader(
    web_paths=("https://github.com/JAlcocerT/Streamlit-MultiChat",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [7]:
!pip show langchain-text-splitters

Name: langchain-text-splitters
Version: 0.3.8
Summary: LangChain text splitting utilities
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core
Required-by: langchain


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

Get Ollama ready:

* https://github.com/JAlcocerT/Docker/tree/main/AI_Gen/Ollama


```sh
sudo docker stats ollama
```

In [ ]:
# !docker exec -it ollama sh
#hostname -I 
#--->> 172.20.0.2 
#sudo docker inspect ollama

In [12]:
# !ollama run llama3.2:1b

In [13]:
# !ollama pull all-minilm

In [18]:
# !pip install langchain-chroma

# #https://pypi.org/project/langchain-chroma/

In [17]:
!pip show langchain-chroma

Name: langchain-chroma
Version: 0.2.3
Summary: An integration package connecting Chroma and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: chromadb, langchain-core, numpy
Required-by: 


In [ ]:
###DEPRICATED VERSION OF DOING IT###

# from langchain_community.embeddings import OllamaEmbeddings #https://python.langchain.com/docs/integrations/text_embedding/ollama/
# from langchain_chroma import Chroma

# # local_embeddings = OllamaEmbeddings(model="all-minilm")
# # vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

# from langchain_community.embeddings import OllamaEmbeddings
# #local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.0.12:11434")
# #local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.1.5:11434")
# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")

/tmp/ipykernel_535655/1148073819.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")


In [21]:
# #https://pypi.org/project/langchain-ollama/
# !pip install langchain-ollama==0.3.2

In [22]:
!pip show langchain-ollama

Name: langchain-ollama
Version: 0.3.2
Summary: An integration package connecting Ollama and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core, ollama
Required-by: 


In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

#Using a local embedding model
#https://ollama.com/library/all-minilm

# local_embeddings = OllamaEmbeddings(model="all-minilm")
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.0.12:11434")
# local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.1.5:11434")
local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://0.0.0.0:11434")

In [24]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)


In [25]:
question = "what is the repository about?"
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

In [26]:
context = ' '.join([doc.page_content for doc in retrieved_docs])


In [27]:
context

'Streamlit-MultiChat\n\n\nMany LLMs - One Streamlit Web App\n\n\nOpenAI | Anthropic | Ollama | Groq\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nA custom Streamlit Web App to Chat with the latest LLMs and get a per use cost instead of a fixed monthly price.\nFeatures\nUse many large language models: OpenAI, Anthropic, Open / Local LLM\'s with one Streamlit Web App.\n\nLLM Support\n\nOllama - Open Source Models\nOpenAI - GPT 3.5 / GPT4 / GPT4o / GPT4o-mini\nAnthropic - Claude 3 (Opus / Sonnet) / Claude 3.5\nGroq API - LlaMa models using quick LPU inference\n\n\nExtended explanation\n\nSliDev presentation of the Streamlit-MultiChat\nThis blog post →\nDeploy as per https://github.com/JAlcocerT/Streamlit-MultiChat/tree/main/Z_DeployMe\n\n\n\nDuring the process, I also explored: SliDev PPTs, ScrapeGraph, DaLLe, Streamlit Auth and OpenAI as Custom Agents\nGetting Started\nThe Project is documented here →\n\nClone the repository and Run with your API keys 👇\n  \xa0\n\nOpenAI A

In [29]:
# !pip install -U langchain-ollama
#0.3.1

In [ ]:
#https://python.langchain.com/api_reference/ollama/index.html

# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2:1b")

from langchain_ollama.llms import OllamaLLM

#!ollama pull llama3.2:1b
#!ollama list
llm = OllamaLLM(model="llama3.2:1b", base_url="http://0.0.0.0:11434")
#llm = OllamaLLM(model="llama3.2:1b", base_url="http://192.168.0.12:11434") #workin with LLama3.2
#llm = OllamaLLM(model="all-minilm", base_url="http://0.0.0.0:11434") #not working with this model, as it is a embedding model!!!


response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")

In [20]:
response

'There are several oversold and overbought periods:\n\n- Overbought: The Relative Strength Index (RSI) is above 70 (30-70)\n- Oversold: The Relative Strength Index (RSI) is below 30 (70-30)\n\nThese levels can be used to generate buy or sell signals based on technical analysis strategies.'

In [37]:
# !pip install langchain_openai==0.3.16

In [ ]:
#export OPENAI_API_KEY="your-api-key"
#source .env

In [42]:
import openai
import os

# Configure your OpenAI API key (if not set as environment variable)
# openai.api_key = "YOUR_OPENAI_API_KEY"

try:
    models = openai.models.list()
    model_names = [model.id for model in models.data]
    print("Available OpenAI Models:")
    for name in model_names:
        print(f"- {name}")
except openai.error.AuthenticationError as e:
    print(f"Authentication error: {e}")
    print("Please ensure your OPENAI_API_KEY environment variable is set correctly.")
except openai.error.OpenAIError as e:
    print(f"An OpenAI API error occurred: {e}")

Available OpenAI Models:
- gpt-4o-audio-preview-2024-12-17
- dall-e-3
- dall-e-2
- gpt-4o-audio-preview-2024-10-01
- gpt-4-turbo-preview
- text-embedding-3-small
- gpt-4-turbo
- gpt-4-turbo-2024-04-09
- gpt-4.1-nano
- gpt-4.1-nano-2025-04-14
- gpt-4o-realtime-preview-2024-10-01
- gpt-4o-realtime-preview
- babbage-002
- gpt-4
- text-embedding-ada-002
- chatgpt-4o-latest
- gpt-4o-realtime-preview-2024-12-17
- gpt-4o-mini-audio-preview
- gpt-4o-audio-preview
- o1-preview-2024-09-12
- gpt-4o-mini-realtime-preview
- gpt-4.1-mini
- gpt-4o-mini-realtime-preview-2024-12-17
- gpt-3.5-turbo-instruct-0914
- gpt-4o-mini-search-preview
- gpt-4.1-mini-2025-04-14
- o1
- o1-2024-12-17
- davinci-002
- gpt-3.5-turbo-1106
- gpt-4o-search-preview
- gpt-4-1106-preview
- gpt-3.5-turbo-instruct
- gpt-3.5-turbo
- gpt-4o-mini-search-preview-2025-03-11
- gpt-4-0125-preview
- gpt-4o-2024-11-20
- whisper-1
- gpt-4o-2024-05-13
- o1-pro
- gpt-3.5-turbo-16k
- gpt-image-1
- o1-pro-2025-03-19
- o1-preview
- gpt-4-0613

In [43]:
from langchain_openai.llms import OpenAI

# You'll need your OpenAI API key
# It can be set as an environment variable OPENAI_API_KEY=your_key
# Or passed directly as a parameter:



#llm = OpenAI(model_name="gpt-3.5-turbo-instruct") # Or any other OpenAI model
llm = OpenAI(model_name="gpt-4.1-nano")

response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}""")

In [35]:
!pip show langchain_openai

Name: langchain-openai
Version: 0.3.16
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/jalcocert/Desktop/IT/Data-Chat/.venv/lib/python3.10/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


In [44]:
response

'. The repository is about a Streamlit web app that allows users to chat with multiple large language models (LLMs) from various providers like OpenAI, Anthropic, Ollama, and Groq, and compare costs on a per-use basis.'

## All Together

### LangChain + Chroma + OpenAI

### LangChain + Chroma + Ollama

In [24]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
#from langchain.chains import RetrievalQA

def get_response_from_website_with_chunks(
    ollama_base_url: str,
    embedding_model_name: str,
    llm_model_name: str,
    question: str,
    website_url: str,
    content_class: str = "content-area",
    chunk_size: int = 1200,
    chunk_overlap: int = 100,
    search_kwargs_k: int = 3
) -> str:
    """
    Scrapes a website, creates embeddings using Ollama,
    stores them in Chroma, retrieves relevant chunks, and
    answers a question using an Ollama language model,
    following the provided code chunk structure.

    Args:
        ollama_base_url: The base URL of your Ollama server (e.g., "http://localhost:11434").
        embedding_model_name: The name of the Ollama embedding model (e.g., "all-minilm").
        llm_model_name: The name of the Ollama language model to use (e.g., "llama3.2:1b").
        question: The question you want to ask about the website content.
        website_url: The URL of the website to process.
        content_class: The CSS class name of the main content area on the website.
        chunk_size: The size of each text chunk.
        chunk_overlap: The overlap between adjacent text chunks.
        search_kwargs_k: The number of top documents to retrieve.

    Returns:
        A string containing the answer to the question based on the website content.
    """
    try:
        # Step 1: Web Scraping with LangChain
        bs4_strainer = bs4.SoupStrainer(class_=(content_class))
        loader = WebBaseLoader(
            web_paths=(website_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        docs = loader.load()

        # Step 2: Text Splitting for Manageable Chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True
        )
        all_splits = text_splitter.split_documents(docs)

        # Step 3: Generating Embeddings with Ollama and Chroma
        local_embeddings = OllamaEmbeddings(base_url=ollama_base_url, model=embedding_model_name)
        vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

        # Step 4: Implementing the Retrieval System
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": search_kwargs_k})
        retrieved_docs = retriever.invoke(question)
        context = ' '.join([doc.page_content for doc in retrieved_docs])

        # Step 5: Generating Responses with a Language Model
        llm = OllamaLLM(base_url=ollama_base_url, model=llm_model_name)
        response = llm.invoke(f"""Answer the question according to the context given very briefly:
                   Question: {question}.
                   Context: {context}
        """)
        return response

    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    ollama_url = "http://192.168.0.12:11434"  # Replace with your Ollama server URL if different
    embedding_model = "all-minilm"
    llm_model = "llama3.2:1b"
    user_question = "what are the oversold and overbought periods?"
    target_website = "https://pythonology.eu/using-pandas_ta-to-generate-technical-indicators-and-signals/"
    content_area_class = "content-area" # Use the same class as in the example

    answer = get_response_from_website_with_chunks(
        ollama_url,
        embedding_model,
        llm_model,
        user_question,
        target_website,
        content_class=content_area_class
    )
    print(f"Question: {user_question}")
    print(f"Answer: {answer}")

Question: what are the oversold and overbought periods?
Answer: - Oversold: Below 20 
- Overbought: Above 80


In [45]:
# !pip freeze > requirements-export.txt